In [51]:
import pandas as pd
import numpy as np
#pd.set_option('display.max_rows', 50000)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from sklearn.metrics.pairwise import cosine_similarity
import re

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# todo:
# other library for languages detection ? qualifiers included if exists? 
# duplicated titles: red queen 1, The Secret Garden
# variable ngram
# clean the titles --> DONE
# try out with other languages
# recommend for all the books

In [52]:
#pfad = "drive/MyDrive/DataMiningSeminar/"
items = pd.read_csv("data/items.csv", delimiter='|')
transactions = pd.read_csv("data/transactions.csv", delimiter='|')
evaluation = pd.read_csv("data/evaluation.csv", delimiter='|')
print(items.shape)
print(transactions.shape)
print(evaluation.shape)

(78030, 6)
(365143, 5)
(1000, 1)


In [194]:
items_new = pd.read_csv('items_language.csv')
titles_eng = items_new.loc[items_new.language == "en"]

# Prepocessing title

In [105]:
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

In [147]:
# lowercase
def lower_case(title):
    return np.char.lower(title)
#remove the stop words

def remove_stop_words(title, lang):
    stop_words = set(stopwords.words(lang))
    word_tokens = word_tokenize(title) 
  
    filtered_title = [] 
  
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_title.append(w)
    return filtered_title
# remove punctuation:
def remove_punctuation(title):
    symbols = "!\"#$%&()*+-./:';<=>?@[\]^_`{|}~\n"
    filtered_title = [] 
  
    for t in title: 
        if t not in symbols: 
            filtered_title.append(str(t))
    return filtered_title
# remove single characters
#def remove_single_characters(text):
#    new_text = ""
#    for t in text:
#        if len(t) > 1:
#            new_text = new_text + " " + t
#    return new_text
#stemmer = PorterStemmer()

#def stemming_text(title):
    
#    filtered_title = []
#    for t in title:
#        filtered_title.append(stemmer.stem(t))
#    return filtered_title


In [155]:
titles_eng["title_processed"] = titles_eng.title.apply(lambda x: lower_case(str(x)))
titles_eng["title_processed"] = titles_eng.title_processed.apply(lambda x: remove_stop_words(str(x), "english"))
titles_eng["title_processed"] = titles_eng.title_processed.apply(lambda x: remove_punctuation(x))
#titles_eng["title_processed"] = titles_eng.title_processed.apply(lambda x: remove_single_characters(str(x)))
#titles_eng["title_processed"] = titles_eng.title_processed.apply(lambda x: stemming_text(x))


/Users/thuylinhnguyen/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/thuylinhnguyen/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/thuylinhnguyen/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .l

In [186]:
df = titles_eng.copy()
df['author'] = df['author'].map(lambda x: x.split(',') if str(x) != "nan" else "nan")
df['main topic'] = df['main topic'].map(lambda x: x.split(',') if str(x) != "nan" else "nan")

#df["author"] = df["author"].apply(lambda x: i.lower().replace(' ','') for i in x)
l = []
for index, row in df.iterrows():
    l.append([x.lower().replace(' ','') for x in row['author']])
df["author"] = l
#    row['main topic'] = [x.lower().replace(' ','') for x in row['main topic']]

In [187]:
l = []
columns = ['main topic', 'author', 'title_processed']
for index, row in df.iterrows():
    words = ''
    for col in columns:
        if str(row[col]) != "nan":
            words += ' '.join(row[col]) + ' '
    l.append(words)
df["Bag_of_words"] = l    
df = df[['title','Bag_of_words']]

In [188]:
df.head()

,title,Bag_of_words
0,Princess Poppy: The Big Mix Up,YFB janeylouisejones princess poppi big mix
2,Red Queen 1,YFH victoriaaveyard red queen 1
21,Wir spielen Kasperltheater,ATDH petermitschitczek wir spielen kasperltheat
23,Harry Potter 4 and the Goblet of Fire,YFH joannek.rowling harri potter 4 goblet fire
25,A Vow So Bold and Deadly,YFHR brigidkemmerer vow bold deadli


In [189]:
count = CountVectorizer()
count_matrix = count.fit_transform(df['Bag_of_words'])
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [78]:
#def cosine_similarity_matrix(df, lang):
#    #Converting the book title into vectors and used bigram
#    tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df = 1, stop_words=lang)
#    tfidf_matrix = tf.fit_transform(df['title'])
#    cosine_similarities = cosine_similarity(tfidf_matrix, tfidf_matrix) 

In [79]:
#cosine_similarity_matrix(titles_eng, 'english')

In [190]:
def recommendations(title, cosine_sim = cosine_similarities):
    # initializing the empty list of recommended movies
    recommendations = [] 
    indices = pd.Series(df['title'])
    
    # gettin the index of the book that matches the title
    idx = indices[indices == title].index[0]
    
    ##if title not in idx:
    ##    raise KeyError("title is not in indices")
    
    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indices of the 5 most similar books
    top_5_indices = list(score_series.iloc[1:6].index)

    # populating the list with the titles of the best 5 matching books
    for i in top_5_indices:
        recommendations.append(list(df["title"])[i])

    return recommendations

In [193]:
recommendations("Anne of Green Gables")

['Doughnut Street School And The Mystery Of The Doughnut Hole',
 "The Inventor's Clone",
 'The Roman, the Twelve & the King',
 'The 13th Apostle',
 'A Sword Named Truth']

In [181]:
titles_eng.title.value_counts()

The Secret Garden                                                                                       25
Anne of Green Gables                                                                                    19
Alice's Adventures in Wonderland                                                                        18
The Railway Children                                                                                    17
A Christmas Carol                                                                                       14
                                                                                                        ..
Fancy Nancy: Nancy Clancy, Super Sleuth                                                                  1
Chroniken von Chaos und Ordnung Band 1-4: Thorn Gandir / Telos Malakin / Bargh Barrowson / Lucretia      1
Shard Abyss                                                                                              1
The Secret Message                   

In [ ]:
'''def recommend(title):# Convert the index into series
    indices = pd.Series(titles_eng.index, index = titles_eng['title'])
    
    #Converting the book title into vectors and used bigram
    tf = TfidfVectorizer(analyzer='word', ngram_range=(2, 2), min_df = 1, stop_words='english')
    tfidf_matrix = tf.fit_transform(titles_eng['title'])
    
    # Calculating the similarity measures based on Cosine Similarity
    sg = cosine_similarity(tfidf_matrix, tfidf_matrix)
    
    # Get the index corresponding to original_title
    idx = indices[title]
    # Get the pairwsie similarity scores 
    sig = list(enumerate(sg[idx]))
    # Sort the books
    sig = sorted(sig, key=lambda x: x[1], reverse=True)
    # Scores of the 5 most similar books 
    sig = sig[1:6]
    # Book indicies
    book_indices = [i[0] for i in sig]
   
    # Top 5 book recommendation
    rec = titles_eng[['title']].iloc[book_indices]
    print(rec)'''